In [1]:
from sympy.combinatorics import *
from itertools import combinations
from sympy.interactive import init_printing

from tqdm.notebook import tqdm

from math import factorial
import numpy as np

init_printing(perm_cyclic=True, pretty_print=False) #prints permutations in compact notation
perm_type = permutations.Permutation

# Reproduce (correct) coefficients from project catalog

In [2]:
def gen_ctype(l: dict,n: int):
    '''
    Input:
        l - cycle decompostion as a dictionary including 1 cycles. e.g. {1:2, 2:1} is a transposition in S_4
        n - specifies order of symmetric group
    Output:
        Generates all permutations of specified cycletype as a list
    '''
    
    has_to_be_partition = 0
    for key, value in l.items():
        has_to_be_partition = has_to_be_partition + key*value
    if has_to_be_partition < n:
        print("Invalid cycle type format.")
    else:
        list_with_cycle_type = []
        for i in list(symmetric(n)):
            if i.cycle_structure == l:
                list_with_cycle_type.append(i)            
        return list_with_cycle_type
    
Permutation(3)(0,1).cycle_structure, gen_ctype({1:2,2:1},4)

({1: 2, 2: 1}, [(2 3), (3)(1 2), (1 3), (3)(0 1), (3)(0 2), (0 3)])

In [3]:
# Create element g2 as a list of permutations (sympy permutations)
g2 = gen_ctype({1:2, 2:1},4)
g2

[(2 3), (3)(1 2), (1 3), (3)(0 1), (3)(0 2), (0 3)]

In [4]:
# Generate powers of g2
g2_square = [s*t for s in g2 for t in g2]
g2_cube = [s*t for s in g2_square for t in g2]
g2_quad = [s*t for s in g2_cube for t in g2]

g2_square[0:5]

[(3), (1 2 3), (1 3 2), (0 1)(2 3), (0 2 3)]

In [5]:
def memoize(f):
    memo = {}
    def helper(x):
        if x not in memo:            
            memo[x] = f(x)
        return memo[x]
    return helper

In [6]:
@memoize
def ctype(p: perm_type):
    '''
    Input:
        Sympy permutation
    Output:
        Gives cycletype of permutation as a tuple with i'th coordinate being multiplicity of i-cycles.
    '''
    d = p.cycle_structure
    max_val = max(d.keys())
    L = []
    for i in range(1,max_val+1):
        if i in list(d.keys()):
            L.append(d[i])
        else:
            L.append(0)
    return tuple(L)

ctype(Permutation(3)(1,3)) # has 2 1-cycles (0 and 2) and one 2-cycle

(2, 1)

In [22]:
def decompose(plist: list):
    '''
    Input: List of permutations
    Output: Decomposes list by occurences of cycletype
    '''
    cycle_count = {}
    for p in tqdm(plist):
        p_type = ctype(p)
        if p_type in list(cycle_count.keys()):
            cycle_count[p_type] += 1
        else:
            cycle_count[p_type] = 1
    return cycle_count

print("g2 squared decomp", decompose(g2_square),
      "\n" "g2 cubed decomp", decompose(g2_cube))


g2 squared decomp {(5,): 10, (2, 0, 1): 60, (1, 2): 30} 
g2 cubed decomp {(3, 1): 340, (1, 0, 0, 1): 480, (0, 1, 1): 180}


In [18]:
def num_ctype(ctuple):
    '''
    Input: cycle type as a tuple
    Output: number of permutations with given cycle type using 
    '''
    n = sum((i+1)*ctuple[i] for i in range(len(ctuple)))
    numerator = factorial(n)
    denominator = 1
    for i in range(len(ctuple)):
        denominator *= ((i+1)**ctuple[i]) * factorial(ctuple[i])
    return numerator // denominator

num_ctype((2,1))

6

In [19]:
def decompose2(plist):
    '''
    Input: list of permutations
    Output: decomposition according to A(d)
    '''
    d = decompose(plist)
    for i in d.keys():
        d[i] //= num_ctype(i)
    return d

In [20]:
print("g2 square", decompose2(g2_square), 
      "\n" "g2 cube", decompose2(g2_cube), 
      "\n" "g2 quad", decompose2(g2_quad))

g2 square {(5,): 10, (2, 0, 1): 3, (1, 2): 2} 
g2 cube {(3, 1): 34, (1, 0, 0, 1): 16, (0, 1, 1): 9} 
g2 quad {(5,): 340, (2, 0, 1): 207, (1, 2): 168, (0, 0, 0, 0, 1): 125}


### Try with d = 5

In [11]:
g2 = gen_ctype({1:3, 2:1},5)
g2

[(3 4), (4)(2 3), (2 4), (4)(1 2), (4)(1 3), (1 4), (4)(0 1), (4)(0 2), (4)(0 3), (0 4)]

In [12]:
g2_square = [s*t for s in g2 for t in g2]
g2_cube = [s*t for s in g2_square for t in g2]
g2_quad = [s*t for s in g2_cube for t in g2]

In [13]:
print("g2 square", decompose2(g2_square), 
      "\n" "g2 cube", decompose2(g2_cube), 
      "\n" "g2 quad", decompose2(g2_quad))

g2 square {(5,): 10, (2, 0, 1): 3, (1, 2): 2} 
g2 cube {(3, 1): 34, (1, 0, 0, 1): 16, (0, 1, 1): 9} 
g2 quad {(5,): 340, (2, 0, 1): 207, (1, 2): 168, (0, 0, 0, 0, 1): 125}


## Output coefficient matrix

In [21]:
d = 5
g2 = gen_ctype({1:(d-2), 2:1}, d)

g2_powers = [[p*q for p in g2 for q in g2]]

for i in tqdm(range(4)):
    g2_powers.append([p*q for p in g2 for q in g2_powers[-1]])

In [23]:
g2_powers_decomp = []
for i in tqdm(g2_powers):
    g2_powers_decomp.append(decompose2(i))

In [24]:
all_keys = set([])
for decomp in g2_powers_decomp:
    all_keys = all_keys.union(set(decomp.keys()))
all_keys = all_keys - {(d-2, 1), (d,)}
all_keys = list(all_keys)
all_keys.sort()
all_keys

[(0, 0, 0, 0, 1), (0, 1, 1), (1, 0, 0, 1), (1, 2), (2, 0, 1)]

In [25]:
(n,m) = (len(g2_powers_decomp),len(all_keys))
A = np.zeros((n,m)).astype(int)

for i in range(n):
    for j in range(m):
        try:
            A[i,j] = (g2_powers_decomp[i])[all_keys[j]]
        except:
            pass

A

array([[    0,     0,     0,     2,     3],
       [    0,     9,    16,     0,     0],
       [  125,     0,     0,   168,   207],
       [    0,  1461,  1664,     0,     0],
       [15625,     0,     0, 16672, 17703]])

In [26]:
np.linalg.matrix_rank(A)

5

# Trashcan

In [ ]:
def test_ctype(p: perm_type, D):
    '''
    Input:
        p
    '''
    d = p.cycle_structure
    d.pop(1, None)
    if d.keys() == D.keys():
        for key in D.keys():
            if d[key] != D[key]:
                return False
        return True
    return False

In [ ]:
list_of_cycles_with_trans = []
for i in list(symmetric(3)):
    try:
        i.cycle_structure[2]
        list_of_cycles_with_trans.append(i)
    except:
        print("Cycle not found")

list_of_cycles_with_trans

test_dict = {2:2}
test_dict, Permutation(0,1)(2,3).cycle_structure

In [ ]:
type1 = []
type2 = []
type3 = []

for p in g2_square:
    if test_ctype(p, {}):
        type1.append(p)
    if test_ctype(p, {2:2}):
        type3.append(p)
    if test_ctype(p, {3:1}):
        type2.append(p)

In [ ]:
len((1,2,2,3,3))

In [ ]:
num_ctype(4, (0,0,0,1))

In [ ]:
a,b = Permutation(3)(0,1),Permutation(3)(1,0)

In [ ]:
len(g2_powers[-1])